## Implementing SVMs

In [1]:
# Import Statements
import numpy as np
import cvxopt
import os
import math
from sklearn.datasets import load_svmlight_file
from cvxopt import matrix, solvers

### Training Data

The binary classification dataset has 60 input features
- the training set has 1000 samples
- the test set has 2173 samples

In [2]:
'''
Example for reading data and the corresponding labels
'''
# loading a dataset from a file in the LIBSVM format 
# The first row contains the class label, in this case 0 or 1.
# Following that are the features, here there are two values for each one; the first one is the feature index (i.e. which feature it is) and the second one is the actual value

DATA_DIR = '.' # sets variable to current directory
data_train, label_train = load_svmlight_file(os.path.join(DATA_DIR, 'train.txt'),n_features=60) 

In [3]:
data_test, label_test = load_svmlight_file(os.path.join(DATA_DIR, 'test.txt'),n_features=60) 

data_train will have the data in sparse matrix form: \
(0, 0)	2.0   -> in the 0th row, 0th col, the value is 2\
(0, 1)	1.0   -> in the 0th row, 1st col, the value is 1

### 5.1 Data Preprocessing

xk -> one column\
N = 1000

In [4]:
# Computing the mean
num_features = data_train.get_shape()[1]
N = data_train.get_shape()[0]
mean_of_features = []

for i in range(num_features):
    column = data_train[:, i]
    sum_of_columns = np.sum(column)
    mean_of_features.append((1/N) * sum_of_columns)


In [5]:
# Subtracting the mean from all values of the feature
data_train = data_train - np.array(mean_of_features)

In [6]:
# Calculating standard deviation of each feature
sd_of_features = []

for i in range(num_features):
    column = data_train[:, i]
    column = np.square(column)
    sum_of_columns = np.sum(column)
    sd_term = (1/(N-1)) * sum_of_columns
    sd_of_features.append(math.sqrt(sd_term))

In [7]:
# Divide each feature by its standard deviation
for i in range(num_features):
    column = data_train[:, i]
    data_train[:,i] = column / sd_of_features[i]

In [8]:
# Getting the 2nd and 9th feature's mean and standard deviation:
print("For the training dataset:")
print(f"The 2nd feature's mean is: {mean_of_features[1]} and the 2nd feature's SD is {sd_of_features[1]}")
print(f"The 9th feature's mean is: {mean_of_features[8]} and the 9th feature's SD is {sd_of_features[8]}")

For the training dataset:
The 2nd feature's mean is: 2.46 and the 2nd feature's SD is 1.1223456061568822
The 9th feature's mean is: 2.537 and the 9th feature's SD is 1.1062797362579027


In [9]:
#TODO: Apply mean and SD to test dataset too
data_test = data_test - np.array(mean_of_features)

num_features_test = data_test.shape[1]
for i in range(num_features):
    column = data_test[:, i]
    data_test[:,i] = column / sd_of_features[i]

### 5.2 Implement Linear SVM

In [10]:
def train_svm(train_data, train_label, C):
  """Train linear SVM (primal form). This is basically solving the primal equation.

  Argument:
    train_data: N*D matrix, each row as a sample and each column as a feature
    train_label: N*1 vector, each row as a label
    C: tradeoff parameter (on slack variable side)

  Return:
    w: feature vector (column vector)
    b: bias term
  """
  N = train_data.shape[0] # num rows
  d = train_data.shape[1] # num cols

  # P, q - coefficients of the quadratic and linear 
  P = matrix(np.diag(np.concatenate([[0], [1] * d, [0] * N])), tc='d') #bias (no quadratic), w (1 for each quad), slack (no quad)
  q = matrix(np.array(np.concatenate([[0],[0]*d,[C]*N])), tc='d') # bias (no linear), w (no linear), slack (C)

  # Creating G - coefficients of inequality
  G = np.zeros((N+N, N+d+1)) # The first N rows are for the first constraint -> b, w, slack. Next N rows are for the second constraint for the slack

  # constraint 1 -> -yiw^Txi - yib - slack <= -1
  for row in range(N):
    G[row,0] = -train_label[row] # 0th column is the bias
    G[row,1:d+1] = -train_label[row] * train_data[row] # first to 60th columns are the weights
    G[row,d+1+row] = -1 # filling in an identity matrix for the slack

  # constraint 2 -> -slack <= 0
  for row in range(N):
    G_index = row+N
    G[G_index,d+1+row] = -1

  G_fin = matrix(G, tc='d')
  
  # Creating h - right side of inequality --> -1 for the first constraint, 0 for the second constraint
  h_array = np.concatenate((np.array([-1]*N), np.array([0]*N)))
  h = matrix(h_array, tc='d')

  sol = solvers.qp(P, q, G_fin, h)

  x = sol['x'] # optimal values of the decision variables
  b = x[0] # bias is the 0th column
  w = np.array(x[1:d+1]) # weight is the first to d + 1 column
  return w,b


In [11]:
def test_svm(test_data, test_label, w, b):
  """Test linear SVM

  Argument:
    test_data: M*D matrix, each row as a sample and each column as a feature
    test_label: M*1 vector, each row as a label
    w: feature vector
    b: bias term

  Return:
    test_accuracy: a float between [0, 1] representing the test accuracy
  """
  # yi = w^T x + b
  n = len(test_label)
  all_predictions = []
  for row in range(test_data.shape[0]):
    yi = np.dot(test_data[row],w) + b
    if yi[0] > 0:
      all_predictions.append(1)
    else:
      all_predictions.append(-1)
  
  num_correct = 0
  for entry in range(len(test_label)):
    if test_label[entry] == all_predictions[entry]:
      num_correct += 1
  
  return num_correct/n
  

### 5.3 Cross Validation for Linear SVM


k -> number of groups that a given data sample is to be split into \
split dataset into k groups of 100 samples each \
each group takes turns being the test data set, the rest is training \
fit model on training set and evaluate on test set \


In [12]:
# creating the c values
C_values = []
for i in [-6, -5, -4, -3, -2, -1, 0, 1, 2, 3]:
    C_values.append(4**i)

cross_val_ranges = {1: (0,100), 2: (101,200), 3:(201,300), 4:(301,400), 5:(401,500), 
                    6:(501,600), 7:(601,700), 8:(701,800), 9:(801,900), 10:(901,1000)}

c_accuracy = [] 
# 10-fold cross val
for c in C_values:
    cross_val_accuracy = []
    for cross_val_idx in range(1,11):
        # print(f"RUN: c = {c} and val = {cross_val_idx}")
        cross_train = data_train
        cross_train_labels = label_train

        # Making the cross-validation test set
        x,y = cross_val_ranges.get(cross_val_idx)
        cross_test = data_train[x:y, :]
        cross_test_labels = label_train[list(range(x,y))]
        # print(f"    cross_test len: {cross_test.shape} and labels: {cross_test_labels.size}")

        # Making the cross validation train set
        delete = list(range(x,y))
        cross_train = np.delete(cross_train, delete, axis=0)
        cross_train_labels = np.delete(cross_train_labels, delete, axis=0)
        # print(f"    cross_train len: {cross_train.shape} and labels: {cross_train_labels.size}")


        # Calling the functions
        w,b= train_svm(cross_train, cross_train_labels, c)
        ta = test_svm(cross_test, cross_test_labels, w, b)
        cross_val_accuracy.append(ta)
    #print(f"HERE, {c}: {cross_val_accuracy}")

    c_accuracy.append(cross_val_accuracy)
    #print("HI")
    #print(c_accuracy)

     pcost       dcost       gap    pres   dres
 0:  2.2795e-01  2.2110e+02  7e+03  3e+00  2e+03
 1:  6.6106e-01 -7.3111e+01  7e+01  3e-02  2e+01
 2:  6.0639e-01 -3.2784e+00  4e+00  1e-03  9e-01
 3:  4.8389e-01  1.5684e-01  3e-01  1e-15  4e-16
 4:  2.2397e-01  1.7783e-01  5e-02  1e-15  1e-16
 5:  2.0610e-01  1.8886e-01  2e-02  7e-16  9e-17
 6:  2.0570e-01  1.8920e-01  2e-02  6e-16  7e-17
 7:  2.0056e-01  1.9199e-01  9e-03  5e-16  1e-16
 8:  1.9909e-01  1.9311e-01  6e-03  5e-16  7e-17
 9:  1.9716e-01  1.9448e-01  3e-03  4e-16  1e-16
10:  1.9641e-01  1.9503e-01  1e-03  4e-16  1e-16
11:  1.9587e-01  1.9545e-01  4e-04  5e-16  2e-16
12:  1.9565e-01  1.9563e-01  2e-05  5e-16  3e-16
13:  1.9564e-01  1.9564e-01  5e-07  6e-16  4e-16
14:  1.9564e-01  1.9564e-01  1e-08  5e-16  4e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  2.2557e-01  2.2756e+02  7e+03  3e+00  2e+03
 1:  6.4863e-01 -6.9478e+01  7e+01  3e-02  2e+01
 2:  5.9506e-01 -3.2981e+00  4e+00  1e-03  9e-0

In [13]:
for i in c_accuracy:
    print(i)

[0.53, 0.5656565656565656, 0.494949494949495, 0.46464646464646464, 0.5555555555555556, 0.5050505050505051, 0.5959595959595959, 0.4444444444444444, 0.48484848484848486, 0.5555555555555556]
[0.72, 0.7878787878787878, 0.797979797979798, 0.7878787878787878, 0.7474747474747475, 0.8181818181818182, 0.8585858585858586, 0.8080808080808081, 0.797979797979798, 0.8080808080808081]
[0.75, 0.8181818181818182, 0.8080808080808081, 0.8181818181818182, 0.7373737373737373, 0.7878787878787878, 0.8585858585858586, 0.8484848484848485, 0.8080808080808081, 0.8383838383838383]
[0.82, 0.8484848484848485, 0.8080808080808081, 0.8282828282828283, 0.7575757575757576, 0.7878787878787878, 0.8282828282828283, 0.8585858585858586, 0.8282828282828283, 0.8282828282828283]
[0.79, 0.8383838383838383, 0.7575757575757576, 0.8585858585858586, 0.7575757575757576, 0.8080808080808081, 0.8282828282828283, 0.8484848484848485, 0.8181818181818182, 0.8080808080808081]
[0.77, 0.8181818181818182, 0.7676767676767676, 0.8484848484848485,

In [14]:
c_accuracy_matrx = np.array(c_accuracy)

In [15]:
print(c_accuracy_matrx)

[[0.53       0.56565657 0.49494949 0.46464646 0.55555556 0.50505051
  0.5959596  0.44444444 0.48484848 0.55555556]
 [0.72       0.78787879 0.7979798  0.78787879 0.74747475 0.81818182
  0.85858586 0.80808081 0.7979798  0.80808081]
 [0.75       0.81818182 0.80808081 0.81818182 0.73737374 0.78787879
  0.85858586 0.84848485 0.80808081 0.83838384]
 [0.82       0.84848485 0.80808081 0.82828283 0.75757576 0.78787879
  0.82828283 0.85858586 0.82828283 0.82828283]
 [0.79       0.83838384 0.75757576 0.85858586 0.75757576 0.80808081
  0.82828283 0.84848485 0.81818182 0.80808081]
 [0.77       0.81818182 0.76767677 0.84848485 0.73737374 0.78787879
  0.83838384 0.83838384 0.81818182 0.80808081]
 [0.78       0.82828283 0.78787879 0.83838384 0.74747475 0.78787879
  0.83838384 0.82828283 0.81818182 0.80808081]
 [0.79       0.82828283 0.80808081 0.84848485 0.74747475 0.7979798
  0.82828283 0.80808081 0.81818182 0.81818182]
 [0.79       0.82828283 0.80808081 0.84848485 0.74747475 0.7979798
  0.82828283 0

#### averaged accuracy

In [16]:
column_means = np.mean(c_accuracy_matrx, axis=1)

In [17]:
for i in range(len(C_values)):
    print(f"c is {C_values[i]} and average accuracy is: {column_means[i]}")

c is 0.000244140625 and average accuracy is: 0.5196666666666665
c is 0.0009765625 and average accuracy is: 0.7932121212121211
c is 0.00390625 and average accuracy is: 0.8073232323232323
c is 0.015625 and average accuracy is: 0.8193737373737374
c is 0.0625 and average accuracy is: 0.8113232323232322
c is 0.25 and average accuracy is: 0.8032626262626262
c is 1 and average accuracy is: 0.8062828282828283
c is 4 and average accuracy is: 0.8093030303030304
c is 16 and average accuracy is: 0.8093030303030304
c is 64 and average accuracy is: 0.8093030303030304


When c = 0.015625, the average accuracy is the highest at 0.81937 which means this is the optimal c.

#### SVM accuracy on test

In [18]:
w,b = train_svm(data_train,label_train, C=0.015625)
ta = test_svm(data_test, label_test, w, b) 
print(ta)
print(f"The best SVM accuracy is {ta* 100:.2f}%")

     pcost       dcost       gap    pres   dres
 0:  3.9087e+00  2.8531e+02  8e+03  3e+00  2e+03
 1:  3.4417e+01 -1.1353e+02  2e+02  5e-02  3e+01
 2:  2.4353e+01 -4.8375e+00  3e+01  5e-03  3e+00
 3:  8.2815e+00  5.5608e+00  3e+00  1e-04  8e-02
 4:  7.5198e+00  6.2264e+00  1e+00  5e-05  3e-02
 5:  7.1095e+00  6.5415e+00  6e-01  2e-05  1e-02
 6:  6.9361e+00  6.6838e+00  3e-01  6e-06  4e-03
 7:  6.8469e+00  6.7557e+00  9e-02  8e-07  5e-04
 8:  6.8129e+00  6.7820e+00  3e-02  2e-07  1e-04
 9:  6.7978e+00  6.7945e+00  3e-03  3e-09  2e-06
10:  6.7962e+00  6.7959e+00  3e-04  2e-10  1e-07
11:  6.7960e+00  6.7960e+00  6e-06  3e-12  2e-09
Optimal solution found.
0.8464367816091954
The best SVM accuracy is 84.64%
